In [1]:
import os
import filecmp

In [2]:
team_code = [dir for dir in os.listdir(".") if dir.startswith('0')] + ['999']

In [3]:
team_code.sort()

In [4]:
os.system('mkdir 011/robot2/')
os.system('touch 011/robot2/robot2.py')
os.system('git add 011/robot2/robot2.py')
os.system('git mv 051/rcj_soccer_KAREN/rcj_soccer_player_KAREN.py 051/rcj_soccer_KAREN/rcj_soccer_player_KAREN1.py')
os.system('git mv 051/rcj_soccer_KAREN/ 051/rcj_soccer_KAREN1')
os.system('git mv 051/rcj_soccer_player_LUISA/rcj_soccer_player_LUISA.py 051/rcj_soccer_player_LUISA/rcj_soccer_player_LUISA2.py')
os.system('git mv 051/rcj_soccer_player_LUISA/ 051/rcj_soccer_player_LUISA2')
os.system('git mv 051/rcj_soccer_player_OREO/rcj_soccer_player_OREO.py 051/rcj_soccer_player_OREO/rcj_soccer_player_OREO3.py')
os.system('git mv 051/rcj_soccer_player_OREO/ 051/rcj_soccer_player_OREO3')

0

In [172]:
def visibly_exec(cmd):
    print(cmd)
    os.system(cmd)

def any_in(list, line):
    found = False
    for l in list:
        if l in line:
            found = True
    return found

def validate_team(team_dir, attempt_fix=False):
    # Warn about missing logo.png-files
    if not os.path.isfile(team_dir + os.sep + 'logo.png'):
        print(str(team_dir) + ' has no logo.png')
    # Create if they didn't yet exist separate folders for team's included code
    team_libs_path = team_dir + os.sep + 'team_libraries'
    if not os.path.isdir(team_dir + os.sep + 'team_libraries'):
        print(team_libs_path + ' does not exist')
        os.mkdir(team_libs_path)
        visibly_exec('touch ' + team_libs_path + os.sep + '__init__.py')
        visibly_exec('git add ' + team_libs_path + os.sep + '__init__.py')
        for i in range(1,4):
            os.mkdir(team_libs_path + os.sep + 'robot' + str(i))
            visibly_exec('touch ' + team_libs_path + os.sep + 'robot' + str(i) + os.sep + '__init__.py')
            visibly_exec('git add ' + team_libs_path + os.sep + 'robot' + str(i) + os.sep + '__init__.py')
    # adapt actual bot code
    original_bot_file_names = []
    for bot_no in range(1,4):
        bot_dir_path = team_dir + os.sep + 'robot' + str(bot_no)
        bot_file_path = bot_dir_path + os.sep + 'robot' + str(bot_no) + '.py'
        # if bot is not already completely in the correct place, move it
        if not os.path.isfile(bot_file_path):
            print(bot_file_path + ' does not exist in the correct way')
            if attempt_fix:
                presumed_bot_n_dir = [team_dir + os.sep + bot_n_dir for bot_n_dir in os.listdir(team_dir) if bot_n_dir[-1] == str(bot_no)][0]
#                 print(presumed_bot_n_dir)
                # create robot1, robot2, robot3 dirs if they don't exist
                if not presumed_bot_n_dir.endswith('robot' + str(bot_no)):
                    print('No proper directory for ' + str(presumed_bot_n_dir) + ' moving')
                    visibly_exec('git mv ' + presumed_bot_n_dir + ' ' + team_dir + os.sep + 'robot' + str(bot_no))
                # if robot directories are correct, correct file naming
                if not os.path.isfile(bot_file_path):
                    print('Proper directory but no proper filename for ' + str(bot_file_path))
                    proper_bot_n_dir = team_dir + os.sep + 'robot' + str(bot_no)
                    presumed_bot_n_filename = [presumed_file for presumed_file in os.listdir(proper_bot_n_dir) if presumed_file[-4:-3] == str(bot_no)][0]
                    original_bot_file_names.append(presumed_bot_n_filename)
                    presumed_bot_n_path = proper_bot_n_dir + os.sep + presumed_bot_n_filename
                    proper_bot_n_path = team_dir + os.sep + 'robot' + str(bot_no) + os.sep + 'robot' + str(bot_no) + '.py'
                    visibly_exec('git mv ' + presumed_bot_n_path + ' ' + proper_bot_n_path)
        # move local includes to team_libraries and adapt import statements accordingly
        if not os.listdir(bot_dir_path) == ['robot' + str(bot_no) + '.py']:
            robot_n_py_path = bot_dir_path + os.sep + 'robot' + str(bot_no) + '.py'
            robot_n_py_fd = open(robot_n_py_path, 'r')
            robot_n_py = robot_n_py_fd.readlines()
            robot_n_py_fd.close()
            robot_n_py_fd = robot_n_py_fd = open(bot_dir_path + os.sep + 'robot' + str(bot_no) + '.py', 'w')
            robot_n_py_fd.write("PACKAGE_PARENT = '..'\n")
            bot_extraneous_files = [bot_file for bot_file in os.listdir(bot_dir_path) if bot_file != 'robot' + str(bot_no) + '.py']
            bot_local_modulenames = [filename[:-3] for filename in bot_extraneous_files]
            original_bot_modulenames = [filename[:-3] for filename in original_bot_file_names]
            import_modulenames_to_move = bot_local_modulenames + original_bot_modulenames
            print(import_modulenames_to_move)
            for bot_local_module in bot_local_modulenames:
                # move module
                local_filename_fullpath = team_dir + os.sep + 'robot' + str(bot_no) + os.sep + bot_local_module + '.py'
                visibly_exec('git mv ' + local_filename_fullpath + ' ' + team_dir + os.sep + 'team_libraries/robot' + str(bot_no) + os.sep)
                # change references
                for line in robot_n_py:
                    if 'import' in line and not line.strip().startswith('#') and any_in(bot_local_modulenames, line):
                        if 'from' in line:
                            import_name = line.split(' ')[1]
                            robot_n_py_fd.write('from ..team_' + team_dir + '_libraries.robot' + str(bot_no) + '.' + import_name + ' ' + ' '.join(line.split(' ')[2:]) + '\n')
                        else:
                            import_names = line.replace('import ','').split(',')
                            import_names = [import_name.strip() for import_name in import_names]
                            robot_n_py_fd.write('from ..team_' + team_dir + '_libraries.robot' + str(bot_no) + ' import ' + ','.join(import_names) + '\n')
                    else:
                        if 'import' in line:
                            print(line)
                        robot_n_py_fd.write(line)
            robot_n_py_fd.close()
            visibly_exec('git add ' + robot_n_py_path)
#             visibly_exec('cp ' + bot_file_path + ' ' + team_dir + os.sep + 'team_libraries/robot' + str(bot_no) + os.sep)
#             visibly_exec('git add ' + team_dir + os.sep + 'team_libraries/robot' + str(bot_no) + os.sep)

# OLD CODE
#             print(str(os.listdir(bot_dir_path)) + ' are too many things')
#             for bot_file in bot_extraneous_file:
#                 source_file_path = bot_dir_path + os.sep + bot_file
#                 target_file_path = team_dir + os.sep + 'team_libraries' + os.sep + 'robot' + str(bot_no) + os.sep + bot_file
#                 if not os.path.isfile(target_file_path):
#                     1 == 1
#                     visibly_exec('git mv ' + source_file_path + ' ' + target_file_path)
#                 else:
#                     print(target_file_path + ' already exists')
#                     if filecmp.cmp(source_file_path, target_file_path):
#                         print('Files are identical')
#                         print("Files are identical. Deleting extraneous copy.")
#                         visibly_exec('git rm -f ' + source_file_path)
#                     else:
#                         print("Files are different: diff " + source_file_path + ' ' + target_file_path)

In [254]:
def visibly_exec(cmd):
    print(cmd)
    os.system(cmd)

def any_in(list, line):
    found = False
    for l in list:
        if l in line:
            found = True
    return found

def validate_team(team_dir, attempt_fix=False):
    # Warn about missing logo.png-files
    if not os.path.isfile(team_dir + os.sep + 'logo.png'):
        print(str(team_dir) + ' has no logo.png')
    # Create if they didn't yet exist separate folders for team's included code
    team_libs_path = team_dir + os.sep + 'team_libraries'
    if not os.path.isdir(team_dir + os.sep + 'team_libraries'):
        print(team_libs_path + ' does not exist')
        os.mkdir(team_libs_path)
        visibly_exec('touch ' + team_libs_path + os.sep + '__init__.py')
        visibly_exec('git add ' + team_libs_path + os.sep + '__init__.py')
        for i in range(1,4):
            os.mkdir(team_libs_path + os.sep + 'robot' + str(i))
            visibly_exec('touch ' + team_libs_path + os.sep + 'robot' + str(i) + os.sep + '__init__.py')
            visibly_exec('git add ' + team_libs_path + os.sep + 'robot' + str(i) + os.sep + '__init__.py')
    # adapt actual bot code
    original_bot_file_names = []
    for bot_no in range(1,4):
        bot_dir_path = team_dir + os.sep + 'robot' + str(bot_no)
        bot_file_path = bot_dir_path + os.sep + 'robot' + str(bot_no) + '.py'
        # if bot is not already completely in the correct place, move it
#         if not os.path.isfile(bot_file_path):
        if True: #always execute, relocation is also necessary on bots with correct filenames
            print(bot_file_path + ' does not exist in the correct way')
            if attempt_fix:
                presumed_bot_n_dir = [team_dir + os.sep + bot_n_dir for bot_n_dir in os.listdir(team_dir) if bot_n_dir[-1] == str(bot_no)][0]
#                 print(presumed_bot_n_dir)
                # create robot1, robot2, robot3 dirs if they don't exist
                if not presumed_bot_n_dir.endswith('robot' + str(bot_no)):
                    print('No proper directory for ' + str(presumed_bot_n_dir) + ' moving')
                    visibly_exec('git mv ' + presumed_bot_n_dir + ' ' + team_dir + os.sep + 'robot' + str(bot_no))
                # if robot directories are correct, correct file naming
                if not os.path.isfile(bot_file_path):
                    print('Proper directory but no proper filename for ' + str(bot_file_path))
                    proper_bot_n_dir = team_dir + os.sep + 'robot' + str(bot_no)
                    presumed_bot_n_filename = [presumed_file for presumed_file in os.listdir(proper_bot_n_dir) if presumed_file[-4:-3] == str(bot_no)][0]
                    original_bot_file_names.append(presumed_bot_n_filename)
                    presumed_bot_n_path = proper_bot_n_dir + os.sep + presumed_bot_n_filename
                    proper_bot_n_path = team_dir + os.sep + 'robot' + str(bot_no) + os.sep + 'robot' + str(bot_no) + '.py'
                    visibly_exec('git mv ' + presumed_bot_n_path + ' ' + proper_bot_n_path)
        # move local includes to team_libraries and adapt import statements accordingly
        if not os.listdir(bot_dir_path) == ['robot' + str(bot_no) + '.py']:
            robot_n_py_path = bot_dir_path + os.sep + 'robot' + str(bot_no) + '.py'
            robot_n_py_fd = open(robot_n_py_path, 'r')
            robot_n_py = robot_n_py_fd.readlines()
            robot_n_py_fd.close()
            robot_n_py_fd = robot_n_py_fd = open(bot_dir_path + os.sep + 'robot' + str(bot_no) + '.py', 'w')
            robot_n_py_fd.write("import sys\nsys.path.append('/app/controllers/')\nsys.path.append('.')")
            bot_extraneous_files = [bot_file for bot_file in os.listdir(bot_dir_path) if bot_file != 'robot' + str(bot_no) + '.py']
            bot_local_modulenames = [filename[:-3] for filename in bot_extraneous_files]
            original_bot_modulenames = [filename[:-3] for filename in original_bot_file_names]
            import_modulenames_to_move = bot_local_modulenames + original_bot_modulenames
            print(import_modulenames_to_move)
            for bot_local_module in bot_local_modulenames:
                # move module
                local_filename_fullpath = team_dir + os.sep + 'robot' + str(bot_no) + os.sep + bot_local_module + '.py'
                visibly_exec('git mv ' + local_filename_fullpath + ' ' + team_dir + os.sep + 'team_libraries/robot' + str(bot_no) + os.sep)
                # change references
                for line in robot_n_py:
                    if 'import' in line and not line.strip().startswith('#') and any_in(bot_local_modulenames, line):
                        if 'from' in line:
                            import_name = line.split(' ')[1]
                            robot_n_py_fd.write('from team_' + team_dir + '_libraries.robot' + str(bot_no) + '.' + import_name + ' ' + ' '.join(line.split(' ')[2:]) + '\n')
                        else:
                            import_names = line.replace('import ','').split(',')
                            import_names = [import_name.strip() for import_name in import_names]
                            robot_n_py_fd.write('from team_' + team_dir + '_libraries.robot' + str(bot_no) + ' import ' + ','.join(import_names) + '\n')
                    else:
#                         if 'import' in line:
#                             print(line)
                        robot_n_py_fd.write(line)
            robot_n_py_fd.close()
            robot_n_py_fd = open(robot_n_py_path, 'r')
            robot_n_py = robot_n_py_fd.readlines()
            robot_n_py_fd.close()
            robot_n_py_fd = robot_n_py_fd = open(bot_dir_path + os.sep + 'robot' + str(bot_no) + '.py', 'w')
            # probably a very bad idea, should likely be integrated in loop up there
            for line in robot_n_py:
                line_written = False
                for original_bot_modulename in original_bot_modulenames:
                    if 'from' in line:
                        robot_n_py_fd.write(line.replace(original_bot_modulename, 'team_' + team_dir + '_libraries.robot' + str(bot_no)))
                        line_written = True
                        break
                    else:
                        robot_n_py_fd.write(line)
                        print("No Idea: " + line)
                        line_written = True
                        break
                if not line_written:
                    robot_n_py_fd.write(line)
            robot_n_py_fd.close()
            visibly_exec('git add ' + robot_n_py_path)
        # convert imports in imports to relative imports where appropriate
        bot_n_include_dir = team_dir + os.sep + 'team_libraries/robot' + str(bot_no) + os.sep
        bot_n_include_files = os.listdir(bot_n_include_dir)
        for bot_n_include_file in bot_n_include_files:
            bot_n_include_file_fd = open(bot_n_include_dir + os.sep + bot_n_include_file, 'r')
            bot_n_include_file_lines = bot_n_include_file_fd.readlines()
            bot_n_include_file_fd.close()
            bot_n_include_file_fd = open(bot_n_include_dir + os.sep + bot_n_include_file, 'w')
            local_module_names = [filename[:-3] for filename in bot_n_include_files]
            for line in bot_n_include_file_lines:
                if 'import' in line:
                    if any_in(local_module_names, line):
                        if 'from' in line:
                            line_parts = line.split('from')
                            bot_n_include_file_fd.write(line_parts[0] + 'from .' + line_parts[1].strip() + '\n')
                        else:
                            line_parts = line.split('import')
                            module_list = [s.strip() for s in line_parts[1].split(',')]
                            module_list_unmodified = [module_name for module_name in module_list if not any_in(local_module_names, module_name)]
                            module_list_modified = [module_name for module_name in module_list if any_in(local_module_names, module_name)]
                            if module_list_unmodified:
                                bot_n_include_file_fd.write('import '.join([line_parts[0], ', '.join(module_list_unmodified)]) + '\n')
                            if module_list_modified:
                                bot_n_include_file_fd.write(line_parts[0] + 'from . import ' + ','.join(module_list_modified) + '\n')
                    else:
                        bot_n_include_file_fd.write(line)
                else:
                    bot_n_include_file_fd.write(line)
            bot_n_include_file_fd.close()
        # nice try, but no...
#         for import_file in os.listdir(bot_n_include_dir):
#             fd = open(bot_n_include_dir + os.sep + import_file, 'r')
#             file_contents = fd.read()
#             fd.close()
#             fd = open(bot_n_include_dir + os.sep + import_file, 'w')
#             fd.write('import sys\nsys.path.append(".")\n')
#             fd.write(file_contents)
#             fd.close()

In [257]:
validate_team('041', True)
# for team_dir in team_code:
#     validate_team(team_dir, True)

041/team_libraries does not exist
touch 041/team_libraries/__init__.py
git add 041/team_libraries/__init__.py
touch 041/team_libraries/robot1/__init__.py
git add 041/team_libraries/robot1/__init__.py
touch 041/team_libraries/robot2/__init__.py
git add 041/team_libraries/robot2/__init__.py
touch 041/team_libraries/robot3/__init__.py
git add 041/team_libraries/robot3/__init__.py
041/robot1/robot1.py does not exist in the correct way
['Goalie', 'SupportOptiPos', '.gitign', 'intercepts', 'utils', 'CoordinateRecalculator', 'MovementCalculator', 'rcj_soccer_robot', 'GoToFunc']
git mv 041/robot1/Goalie.py 041/team_libraries/robot1/
git mv 041/robot1/SupportOptiPos.py 041/team_libraries/robot1/
git mv 041/robot1/.gitign.py 041/team_libraries/robot1/
git mv 041/robot1/intercepts.py 041/team_libraries/robot1/
git mv 041/robot1/utils.py 041/team_libraries/robot1/
git mv 041/robot1/CoordinateRecalculator.py 041/team_libraries/robot1/
git mv 041/robot1/MovementCalculator.py 041/team_libraries/robot

In [29]:
same_dir_py_files = os.listdir('001/omicron_agent_1')
same_dir_py_files

['states.py',
 'fsm.py',
 'ipc.py',
 'utils.py',
 'rcj_soccer_robot.py',
 'predictor.py',
 'omicron_agent_1.py']

In [14]:
robot1_py_file = open('001/omicron_agent_1/omicron_agent_1.py', 'r')
robot1_py = robot1_py_file.read()

In [16]:
robot1_py_lines = robot1_py.split('\n')

In [35]:
robot1_py_lines_imports = [line for line in robot1_py_lines if 'import' in line]

In [36]:
robot1_py_lines_imports_straight = [line for line in robot1_py_lines_imports if not 'from' in line]
robot1_py_lines_imports_from = [line for line in robot1_py_lines_imports if 'from' in line]

In [42]:
for line in robot1_py_lines:
    if 'import' in line:
        imported_modules = []
        if not 'from' in line:
            imported_modules = line.replace('import ', '').split(',')
            for imported_module in imported_modules:
                if imported_module + '.py' in same_dir_py_files:
                    line = line.replace(imported_module, '..team_libraries.bot1_folder.' + imported_module)
                    print(line)

import ..team_libraries.bot1_folder.fsm
import ..team_libraries.bot1_folder.states
import ..team_libraries.bot1_folder.utils
import ..team_libraries.bot1_folder.ipc


In [76]:
robot1_py_file = open('001/robot1/robot1.py', 'r')
robot1_py_file.readlines()

['# This file is part of Team Omicron in RoboCup Jr 2021 Soccer Simulation.\n',
 '# Copyright (c) 2021 Ethan Lo & Matt Young. All rights reserved.\n',
 '#\n',
 '# This Source Code Form is subject to the terms of the Mozilla Public\n',
 '# License, v. 2.0. If a copy of the MPL was not distributed with this\n',
 '# file, You can obtain one at http://mozilla.org/MPL/2.0/.\n',
 '# \n',
 '# Main file for Omicron agent\n',
 '# Based on the rcj_soccer_player controller that ships with the simulator.\n',
 '\n',
 'import math\n',
 'from rcj_soccer_robot import RCJSoccerRobot, TIME_STEP\n',
 'import fsm\n',
 'import states\n',
 'import utils\n',
 'from fsm import RobotState, StateMachine\n',
 'import ipc\n',
 '\n',
 'class OmicronAgent(RCJSoccerRobot):\n',
 '    def setup(self):\n',
 '        # TODO consider moving this to an __init__ constructor? will it work?\n',
 '        self.rs = RobotState()\n',
 '        self.rs.agent_name = self.name\n',
 '        self.rs.agent_id = self.player_id\n',
 '

In [84]:
'test.py'[:-3]

'test'

In [200]:
'            import utils'.split('import')

['            ', ' utils']

In [205]:
'import '.join(['         ', ', '.join(['math', 'sys'])])

'         import math, sys'